## 1. Lecture part (unchanged.. almost)

In [1]:
import torch
from torch import nn
import re
import random
import tqdm
import time

In [2]:
!wget https://s3.amazonaws.com/text-datasets/nietzsche.txt

--2024-10-14 02:30:33--  https://s3.amazonaws.com/text-datasets/nietzsche.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.198.0, 52.216.212.56, 52.217.254.48, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.198.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 600901 (587K) [text/plain]
Saving to: ‘nietzsche.txt.5’

nietzsche.txt.5     100%[===================>] 586,82K   444KB/s    in 1,3s    

2024-10-14 02:30:36 (444 KB/s) - ‘nietzsche.txt.5’ saved [600901/600901]



In [3]:
with open('nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('length:', len(text))
text = re.sub('[^a-z ]', ' ', text)
text = re.sub('\s+', ' ', text)

length: 600893


In [4]:
text[:100]

'preface supposing that truth is a woman what then is there not ground for suspecting that all philos'

In [5]:
INDEX_TO_CHAR = sorted(list(set(text)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}

In [6]:
CHAR_TO_INDEX

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [7]:
MAX_LEN = 40
STEP = 3
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(text) - MAX_LEN, STEP):
    SENTENCES.append(text[i: i + MAX_LEN])
    NEXT_CHARS.append(text[i + MAX_LEN])
print('Num sents:', len(SENTENCES))

Num sents: 193075


In [98]:
print('Vectorization in process...', end='\r')
X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)
for i, sentence in enumerate(SENTENCES):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]
print('Vectorization complete', ' ' * 10)

Vectorization complete           


In [9]:
X[0:1], Y[0]

(tensor([[16, 18,  5,  6,  1,  3,  5,  0, 19, 21, 16, 16, 15, 19,  9, 14,  7,  0,
          20,  8,  1, 20,  0, 20, 18, 21, 20,  8,  0,  9, 19,  0,  1,  0, 23, 15,
          13,  1, 14,  0]]),
 tensor(23))

In [10]:
BATCH_SIZE=512
dataset = torch.utils.data.TensorDataset(X, Y)
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)


In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()

        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = nn.Linear(num_hiddens, num_classes)

    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out)
        predictions = self.output(state[0])
        return predictions

In [45]:
model = NeuralNetwork(nn.GRU, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))

In [46]:
with torch.no_grad():
    print(model(X[0:1]))

tensor([[ 0.0432,  0.1230, -0.1100, -0.0770, -0.0945, -0.1421, -0.1470, -0.1433,
         -0.2006, -0.0116, -0.1786, -0.0222,  0.0698,  0.0380, -0.0764,  0.0105,
         -0.0957,  0.1410, -0.0015, -0.0033, -0.0777, -0.0632, -0.0403, -0.1560,
          0.1072, -0.0842, -0.0084]])


In [47]:
model = model.cuda()

In [31]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

def generate_text():
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred.cuda())[0].cpu()
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

In [32]:
generate_text()

acles it is always the degree of wisdom |odxfanxamlgooibuaynbiulkicohgcywlqewncux


In [48]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in tqdm.tqdm_notebook(data):
        X_b, y_b = X_b.cuda(), y_b.cuda()
        optimizer.zero_grad()
        answers = model.forward(X_b)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()


Epoch 0. Time: 5.214, Train loss: 2.104
haps even the most remarkable creation o|nwtred arsing has pore their iss a foll 



Epoch 1. Time: 4.951, Train loss: 1.714
o longer either desired or fully develop|himent the smar sour evil to rumuact let



Epoch 2. Time: 5.267, Train loss: 1.580
mistrust of this species and of its seri|able of that eyceally a but foror fortru



Epoch 3. Time: 5.236, Train loss: 1.502
 today as can be readily disclosed and c|ultated too peop mething every and would



Epoch 4. Time: 5.102, Train loss: 1.449
 such things of pain and ill belong to t|he chlieve wead reasor a man chrisedly d


...


Epoch 95. Time: 5.455, Train loss: 1.028
the distrustful constraint in the commun|ity done of the conduction of refined at



Epoch 96. Time: 5.471, Train loss: 1.034
 creature is far too easily frightened h|imself the permans of an intercise human



Epoch 97. Time: 5.480, Train loss: 1.031
things and how many things which no long|er envicurres to be made calarance and w



Epoch 98. Time: 5.390, Train loss: 1.029
ion to his pupils knowledge for its own |no all misunderstand him for the reasona



Epoch 99. Time: 5.355, Train loss: 1.026
 as to overlook his own interests for he|re as a proudes the feeling of double mo


## 2. Building an LSTM network

### Make some modifications to NeuralNetwork class:

In [63]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()

        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = nn.Linear(num_hiddens, num_classes)

    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out) 
        if type(state) == tuple: ## TRUE IF LSTM
            state = state[0]
#         if isinstance(self.hidden, nn.LSTM): ## ALTERNATIVE WAY
#             state, _ = state
        predictions = self.output(state[0])
        return predictions

### ... and generate_text() function:

In [64]:
def generate_text(model): ##MAKE MODEL A PARAMETER, NOT SOME EXTERNAL VARIABLE
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred.cuda())[0].cpu()
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

### Let's use a smaller dataset and a smaller batch size

In [73]:
BATCH_SIZE=128
dataset = torch.utils.data.TensorDataset(X[:20000], Y[:20000])
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)

### First, let's define and train a GRU model again

In [82]:
model = NeuralNetwork(nn.GRU, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))
optimizer = torch.optim.Adam(model.parameters())

In [83]:
with torch.no_grad():
    print(model(X[0:1]))

tensor([[ 0.2653, -0.1257,  0.1888,  0.1703, -0.0846, -0.1184, -0.0835, -0.1045,
         -0.3232,  0.1249, -0.2949,  0.0492,  0.1707,  0.2678, -0.0229, -0.1215,
         -0.0948,  0.0624, -0.0418, -0.1384,  0.0895,  0.0669,  0.1636, -0.0316,
         -0.2899,  0.2157, -0.2035]])


In [84]:
model.cuda()

NeuralNetwork(
  (embedding): Embedding(27, 64)
  (hidden): GRU(64, 128, batch_first=True)
  (output): Linear(in_features=128, out_features=27, bias=True)
)

In [85]:
for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in tqdm.notebook.tqdm(data):
        X_b, y_b = X_b.cuda(), y_b.cuda()
        optimizer.zero_grad()
        answers = model.forward(X_b)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text(model)

  0%|          | 0/157 [00:00<?, ?it/s]

Epoch 0. Time: 1.240, Train loss: 2.432
 is not infrequently roguish still often|d ros it sert opo rimer the sinc t i on 


  0%|          | 0/157 [00:00<?, ?it/s]

Epoch 1. Time: 1.212, Train loss: 2.068
f it were salvation and glorification as| mend is the the bfryire at the the of p


  0%|          | 0/157 [00:00<?, ?it/s]

...

Epoch 97. Time: 2.160, Train loss: 0.125
may enjoy the approval of all the positi|on of valuaure in evinnerply avounar evi


  0%|          | 0/157 [00:00<?, ?it/s]

Epoch 98. Time: 2.236, Train loss: 0.119
 has his indignation and his sword and t|he avenausion bity dogmate this eagnict 


  0%|          | 0/157 [00:00<?, ?it/s]

Epoch 99. Time: 1.655, Train loss: 0.131
to doubt whether we did not formerly dec|trally every of himself which it be the 


### Now, let's define and train an LSTM model:

In [87]:
model = NeuralNetwork(nn.LSTM, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))
optimizer = torch.optim.Adam(model.parameters())

In [88]:
with torch.no_grad():
    print(model(X[0:1]))

tensor([[ 0.0559, -0.0450, -0.0086,  0.0850,  0.0468,  0.0187,  0.1498, -0.0442,
          0.1511,  0.0475,  0.0316,  0.0083, -0.1111,  0.0467, -0.0089,  0.0675,
         -0.1534, -0.1101,  0.0590,  0.1539, -0.1130,  0.0094,  0.0580, -0.0101,
         -0.0745,  0.0133, -0.0212]])


In [89]:
model = model.cuda()

In [90]:
generate_text(model)

ligion directly or indirectly either as |ciqpwtrqx wmch htcdpwxtxmfakuau zncbgjja


In [92]:
for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in tqdm.notebook.tqdm(data):
        X_b, y_b = X_b.cuda(), y_b.cuda()
        optimizer.zero_grad()
        answers = model.forward(X_b)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text(model)

  0%|          | 0/157 [00:00<?, ?it/s]

Epoch 0. Time: 1.766, Train loss: 1.987
elling christened it intellectual intuit|ion beees fremtittion amtew theind evein


  0%|          | 0/157 [00:00<?, ?it/s]

Epoch 1. Time: 1.782, Train loss: 1.886
running in the race and not to him who h|thim suth ynout st the this wlet not prd


  0%|          | 0/157 [00:00<?, ?it/s]

...

Epoch 97. Time: 2.104, Train loss: 0.198
to its consequences contains the propert|atic be observicion as reall untruth at 


  0%|          | 0/157 [00:00<?, ?it/s]

Epoch 98. Time: 2.216, Train loss: 0.108
 mankind wir scheiden auch hier noch mit|tere of may easely and sencessions as in


  0%|          | 0/157 [00:00<?, ?it/s]

Epoch 99. Time: 2.039, Train loss: 0.087
development of all superstition and also| he ave on the lears of end concestion w
